# Imports

In [1]:
#%pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [2]:
#%pip install plotly

In [3]:
#%pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-ya5ihpif
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-ya5ihpif
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

import numpy as np
import pandas as pd
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
# import plotly.express as px
from torch.utils.data import DataLoader
from typing import Union, List, Optional, Callable, Tuple, Dict, Literal, Set
from jaxtyping import Float, Int
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.utils import to_numpy
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache, patching

import plotly.express as px
import circuitsvis as cv
import os, sys

In [2]:
#%pip install git+https://github.com/neelnanda-io/neel-plotly.git
from neel_plotly import imshow, line, scatter, histogram
import tqdm
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
device

'cuda'

In [4]:
if not os.path.exists("path_patching.py"):
        !wget https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip
        !unzip main.zip 'path_patching-main/ioi_dataset.py'
        !unzip main.zip 'path_patching-main/path_patching.py'
        sys.path.append("/path_patching-main")
        os.remove("main.zip")
        os.rename("/path_patching-main/ioi_dataset.py", "ioi_dataset.py")
        os.rename("/path_patching-main/path_patching.py", "path_patching.py")
        os.rmdir("/path_patching-main")

from path_patching import Node, IterNode, path_patch, act_patch

In [5]:
update_layout_set = {
    "xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat",
    "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor",
    "showlegend", "xaxis_tickmode", "yaxis_tickmode", "xaxis_tickangle", "yaxis_tickangle", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap"
}

def imshow(tensor, return_fig = False, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, **kwargs_pre)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    # things like `xaxis_tickmode` should be applied to all subplots. This is super janky lol but I'm under time pressure
    for setting in ["tickangle"]:
      if f"xaxis_{setting}" in kwargs_post:
          i = 2
          while f"xaxis{i}" in fig["layout"]:
            kwargs_post[f"xaxis{i}_{setting}"] = kwargs_post[f"xaxis_{setting}"]
            i += 1
    fig.update_layout(**kwargs_post)
    fig.show(renderer=renderer)
    if return_fig:
      return fig

def hist(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    names = kwargs_pre.pop("names", None)
    if "barmode" not in kwargs_post:
        kwargs_post["barmode"] = "overlay"
    if "bargap" not in kwargs_post:
        kwargs_post["bargap"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.histogram(x=tensor, **kwargs_pre).update_layout(**kwargs_post)
    if names is not None:
        for i in range(len(fig.data)):
            fig.data[i]["name"] = names[i // 2]
    fig.show(renderer)

In [6]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots

# Model + Dataset Setup

In [60]:
model = HookedTransformer.from_pretrained(
    "gpt-neo-1.3B",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device = device
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt-neo-1.3B into HookedTransformer


In [61]:
clean_format = [
    "When John and Mary went to the shops,{} gave the bag to",
    "When Tom and James went to the park,{} gave the ball to",
    "When Dan and Sid went to the shops,{} gave an apple to",
    "After Martin and Amy went to the park,{} gave a drink to",
    "After Cody and Jay went to the park,{} gave a drink to",
    "After Jack and Jill ran up the hill,{} gave some water to",
    "When Mark and Martin went to the store,{} gave an apple to",
    "When Sarah and Emily went to the library,{} gave a book to",
    "When Cole and George went to the store,{} gave a book to",
    "When Kai and Bryce went to the park,{} gave an apple to",
]
name_pairs = [
    (" Mary", " John"),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
    (" Cody", " Jay"),
    (" Jack", " Jill"),
    (" Mark", " Martin"),
    (" Sarah", " Emily"),
    (" Cole", " George"),
    (" Kai", " Bryce"),
]

# Define 8 prompts
clean_prompts = [
    prompt.format(name)
    for (prompt, names) in zip(clean_format, name_pairs) for name in names[::-1]
]

clean_prompts = [clean_prompts[i] for i in [0, 3, 5, 7, 9, 11, 13, 15, 17, 19]]


# Define the answers for each prompt, in the form (correct, incorrect)
clean_answers = [names[::i] for names in name_pairs for i in (1, -1)]
clean_answers = [clean_answers[i] for i in [0, 3, 5, 7, 9, 11, 13, 15, 17, 19]]


# Define the answer tokens (same shape as the answers)
clean_answer_tokens = torch.concat([
    model.to_tokens(names, prepend_bos=False).T for names in clean_answers
])

# generate 'corrupted ABC prompts'
abc_format = [
    "When Sarah and James went to the shops,{} gave the bag to",
    "When John and Sid went to the park,{} gave the ball to",
    "When Tom and Amy went to the shops,{} gave an apple to",
    "After Dan and Jay went to the park,{} gave a drink to",
    "After Martin and Jill went to the park,{} gave a drink to",
    "After Cody and Martin ran up the hill,{} gave some water to",
    "When Jack and Emily went to the store,{} gave an apple to",
    "When Mark and Mary went to the library,{} gave a book to",
    "When Kai and Bryce went to the store,{} gave a book to",
    "When Cole and George went to the park,{} gave an apple to",

]

abc_name_pairs = [
    (" Cody", " Jay"),
    (" Jack", " Jill"),
    (" Mark", " Martin"),
    (" Sarah", " Emily"),
    (" Mary", " John"),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
    (" Grant", " Mark"),
    (" Chase", " Miles")
]


abc_prompts = [
    prompt.format(name)
    for (prompt, names) in zip(abc_format, abc_name_pairs) for name in names[::-1]
]
abc_prompts = [abc_prompts[i] for i in [0, 3, 5, 7, 9, 11, 13, 15, 17, 19]]


abc_answers = [names[::i] for names in name_pairs for i in (1, -1)]
abc_answers = [abc_answers[i] for i in [0, 3, 5, 7, 9, 11, 13, 15, 17, 19]]

abc_answer_tokens = torch.concat([
    model.to_tokens(names, prepend_bos=False).T for names in abc_answers
])

In [62]:
clean_tokens: Float[Tensor, "batch seq"] = model.to_tokens(clean_prompts)
corrupted_tokens: Float[Tensor, "batch seq"] = model.to_tokens(abc_prompts)

print(clean_tokens.shape)
print(corrupted_tokens.shape)
clean_logits, clean_cache = model.run_with_cache(clean_tokens, prepend_bos = False)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, prepend_bos = False)

clean_answer_tokens, corrupted_answer_tokens = clean_answer_tokens[:, 0], clean_answer_tokens[:, 1]

answer_tokens: Float[Tensor, "batch 2"] = torch.stack((clean_answer_tokens, corrupted_answer_tokens))
answer_tokens = einops.rearrange(answer_tokens, "answer batch -> batch answer")

torch.Size([10, 15])
torch.Size([10, 15])


In [63]:
batch_size, seq_len = clean_tokens.shape[0], clean_tokens.shape[1]

In [64]:
model.reset_hooks()
clean_logits, clean_cache = model.run_with_cache(clean_tokens, prepend_bos = False)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, prepend_bos = False)

In [65]:
def logits_to_ave_logit_diff(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch 2"] = answer_tokens,
    per_prompt: bool = False
):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''
    # Only the final logits are relevant for the answer
    final_logits: Float[Tensor, "batch d_vocab"] = logits[:, -1, :]
    # Get the logits corresponding to the indirect object / subject tokens respectively
    answer_logits: Float[Tensor, "batch 2"] = final_logits.gather(dim=-1, index=answer_tokens)
    # Find logit difference
    correct_logits, incorrect_logits = answer_logits.unbind(dim=-1)
    answer_logit_diff = correct_logits - incorrect_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean().item()

In [66]:
clean_per_prompt_diff = logits_to_ave_logit_diff(clean_logits, per_prompt = True)
corrupted_per_prompt_diff = logits_to_ave_logit_diff(corrupted_logits, per_prompt = True)

#print(logits_to_ave_logit_diff(clean_logits))

clean_average_logit_diff = logits_to_ave_logit_diff(clean_logits)
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits)
print(clean_average_logit_diff)
print(corrupted_average_logit_diff)
print(clean_average_logit_diff-corrupted_average_logit_diff)

4.891021728515625
-0.4926516115665436
5.383673340082169


In [67]:
def noising_ioi_metric(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = clean_average_logit_diff,
    corrupted_logit_diff: float = corrupted_average_logit_diff,
) -> float:
    '''
    Given logits, returns how much the performance has been corrupted due to noising.

    We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
    and -1 when performance has been destroyed (i.e. is same as ABC dataset).
    '''
    #print(logits[-1, -1])
    patched_logit_diff = logits_to_ave_logit_diff(logits)
    return ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

print(f"IOI metric (IOI dataset): {noising_ioi_metric(clean_logits):.4f}")
print(f"IOI metric (ABC dataset): {noising_ioi_metric(corrupted_logits):.4f}")

IOI metric (IOI dataset): 0.0000
IOI metric (ABC dataset): -1.0000


In [68]:
def denoising_ioi_metric(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = clean_average_logit_diff,
    corrupted_logit_diff: float = corrupted_average_logit_diff,
) -> float:
    '''
    We calibrate this so that the value is 1 when performance got restored (i.e. same as IOI dataset),
    and 0 when performance has been destroyed (i.e. is same as ABC dataset).
    '''
    patched_logit_diff = logits_to_ave_logit_diff(logits)
    return ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff) + 1)


print(f"IOI metric (IOI dataset): {denoising_ioi_metric(clean_logits):.4f}")
print(f"IOI metric (ABC dataset): {denoising_ioi_metric(corrupted_logits):.4f}")

IOI metric (IOI dataset): 1.0000
IOI metric (ABC dataset): 0.0000


In [69]:
answer_residual_directions: Float[Tensor, "batch 2 d_model"] = model.tokens_to_residual_directions(answer_tokens)
correct_residual_direction, incorrect_residual_direction = answer_residual_directions.unbind(dim=1)
logit_diff_directions: Float[Tensor, "batch d_model"] = correct_residual_direction - incorrect_residual_direction

In [70]:
def residual_stack_to_logit_diff(
    residual_stack: Float[Tensor, "... batch d_model"],
    cache: ActivationCache,
    logit_diff_directions: Float[Tensor, "batch d_model"] = logit_diff_directions,
) -> Float[Tensor, "..."]:
    '''
    Gets the avg logit difference between the correct and incorrect answer for a given
    stack of components in the residual stream.
    '''
    batch_size = residual_stack.size(-2)
    scaled_residual_stack = residual_stack #cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)
    return einops.einsum(
        scaled_residual_stack, logit_diff_directions,
        "... batch d_model, batch d_model -> ..."
    ) / batch_size

# Helper Functions

In [71]:
def calc_all_logit_diffs(cache):
  clean_per_head_residual, labels = cache.stack_head_results(layer = -1, return_labels = True, apply_ln = True) # per_head_residual.shape = heads batch seq_pos d_model
  # also, for the worried, no, we're not missing the application of LN here since it gets applied in the below function call
  per_head_logit_diff: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache)

  per_head_logit_diff = einops.rearrange(
      per_head_logit_diff,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  correct_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = correct_residual_direction)

  correct_direction_per_head_logit = einops.rearrange(
      correct_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  incorrect_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = incorrect_residual_direction)

  incorrect_direction_per_head_logit = einops.rearrange(
      incorrect_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  return per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit

per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit = calc_all_logit_diffs(clean_cache)

Tried to stack head results when they weren't cached. Computing head results now


In [95]:
def topk_of_Nd_tensor(tensor: Float[Tensor, "rows cols"], k: int):
    '''
    Helper function: does same as tensor.topk(k).indices, but works over 2D tensors.
    Returns a list of indices, i.e. shape [k, tensor.ndim].

    Example: if tensor is 2D array of values for each head in each layer, this will
    return a list of heads.
    '''
    i = torch.topk(tensor.flatten(), k).indices
    return np.array(np.unravel_index(utils.to_numpy(i), tensor.shape)).T.tolist()


In [92]:
top_heads = []
k = 5

flattened_tensor = per_head_logit_diff.flatten().cpu()
_, topk_indices = torch.topk(flattened_tensor, k)
top_layer_arr, top_index_arr = np.unravel_index(topk_indices.numpy(), per_head_logit_diff.shape)

for l, i in zip(top_layer_arr, top_index_arr):
  top_heads.append((l,i))

print(top_heads)

[(14, 13), (20, 6), (22, 2), (23, 8), (18, 6)]


In [96]:
neg_heads = []
neg_indices = torch.nonzero(torch.lt(per_head_logit_diff, -0.1))
neg_heads_list = neg_indices.squeeze().tolist()
for i in neg_heads_list:
  neg_heads.append((i[0], i[1]))

print(neg_heads)

top_neg_heads = topk_of_Nd_tensor(-1 * per_head_logit_diff, 5)

[(0, 0), (0, 2), (0, 3), (0, 4), (0, 6), (0, 8), (0, 9), (0, 11), (0, 15), (1, 1), (1, 8), (1, 10), (1, 11), (1, 14), (2, 2), (2, 6), (2, 11), (2, 14), (2, 15), (3, 5), (3, 8), (3, 11), (3, 14), (4, 4), (4, 10), (4, 11), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 8), (5, 9), (5, 10), (5, 15), (6, 1), (6, 3), (6, 6), (6, 7), (6, 10), (6, 12), (6, 14), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 7), (7, 9), (7, 11), (7, 13), (7, 14), (8, 0), (8, 2), (8, 3), (8, 4), (8, 5), (8, 7), (8, 9), (8, 11), (8, 13), (8, 14), (8, 15), (9, 0), (9, 3), (9, 5), (9, 7), (9, 8), (9, 9), (9, 10), (9, 12), (9, 15), (10, 0), (10, 2), (10, 3), (10, 4), (10, 8), (10, 10), (10, 11), (10, 15), (11, 1), (11, 3), (11, 4), (11, 6), (11, 8), (11, 13), (11, 14), (12, 0), (12, 1), (12, 4), (12, 5), (12, 7), (12, 11), (12, 12), (12, 14), (13, 0), (13, 4), (13, 6), (13, 7), (13, 10), (14, 1), (14, 2), (14, 3), (14, 4), (14, 8), (15, 2), (15, 3), (15, 6), (15, 9), (15, 13), (15, 15), (16, 3), (16, 9), (16, 11)

In [97]:
top_neg_heads

[[15, 6], [20, 7], [17, 8], [22, 12], [19, 10]]

In [74]:
def calc_all_logit_diffs(cache):
  clean_per_head_residual, labels = cache.stack_head_results(layer = -1, return_labels = True) # per_head_residual.shape = heads batch seq_pos d_model
  # also, for the worried, no, we're not missing the application of LN here since it gets applied in the below function call
  per_head_logit_diff: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache)

  per_head_logit_diff = einops.rearrange(
      per_head_logit_diff,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  correct_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = correct_residual_direction)

  correct_direction_per_head_logit = einops.rearrange(
      correct_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  incorrect_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = incorrect_residual_direction)

  incorrect_direction_per_head_logit = einops.rearrange(
      incorrect_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  return per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit

per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit = calc_all_logit_diffs(clean_cache)

In [75]:
def display_all_logits(cache, title = "Logit Contributions", comparison = False, return_fig = False, logits = None):

  a,b,c = calc_all_logit_diffs(cache)
  if logits is not None:
    ld = logits_to_ave_logit_diff(logits)
  else:
    ld = 0.00

  if not comparison:
    fig = imshow(
        torch.stack([a,b,c]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}", "Correct Direction", "Incorrect Direction"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )
  else:

    ca, cb, cc = calc_all_logit_diffs(clean_cache)
    fig = imshow(
        torch.stack([a, b, c, a - ca, b - cb, c - cc]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}", "Correct Direction", "Incorrect Direction", "Logit Diff Diff", "Correction Direction Diff", "Incorrect Direction Diff"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )


  if return_fig:
    return fig



fig = display_all_logits(clean_cache, title = "Logit Contributions on Clean Dataset", return_fig = True, logits = clean_logits)

In [76]:
def stare_at_attention_and_head_pat(cache, layer_to_stare_at, head_to_isolate, display_corrupted_text = False, verbose = True, specific = False, specific_index = 0):
  """
  given a cache from a run, displays the attention patterns of a layer, as well as printing out how much the model
  attends to the S1, S2, and IO token
  """

  tokenized_str_tokens = model.to_str_tokens(corrupted_tokens[0]) if display_corrupted_text else model.to_str_tokens(clean_tokens[0])
  attention_patten = cache["pattern", layer_to_stare_at]
  print(f"Layer {layer_to_stare_at} Head {head_to_isolate} Activation Patterns:")


  if not specific:
    S1 = attention_patten.mean(0)[head_to_isolate][-1][2].item()
    IO = attention_patten.mean(0)[head_to_isolate][-1][4].item()
    S2 = attention_patten.mean(0)[head_to_isolate][-1][10].item()
  else:
    S1 = attention_patten[specific_index, head_to_isolate][-1][2].item()
    IO = attention_patten[specific_index, head_to_isolate][-1][4].item()
    S2 = attention_patten[specific_index, head_to_isolate][-1][10].item()


  print("Attention on S1: " + str(S1))
  print("Attention on IO: " + str(IO))
  print("Attention on S2: " + str(S2))
  print("S1 + IO - S2 = " + str(S1 + IO - S2))
  print("S1 + S2 - IO = " + str(S1 + S2 - IO))
  print("S1 - IO - S2 = " + str(S1 - S2 - IO))


  if verbose:
    display(cv.attention.attention_heads(
      tokens=tokenized_str_tokens,
      attention= attention_patten.mean(0) if not specific else attention_patten[specific_index],
      #attention_head_names=[f"L{layer_to_stare_at}H{i}" for i in range(model.cfg.n_heads)],
    ))
  else:
    print(attention_patten.mean(0).shape)

    display(cv.attention.attention_patterns(
      tokens=tokenized_str_tokens,
      attention=attention_patten.mean(0)if not specific else attention_patten[specific_index],
      attention_head_names=[f"L{layer_to_stare_at} H{i}" for i in range(model.cfg.n_heads)],
    ))

In [44]:
def display_corrupted_clean_logits(cache, title = "Logit Contributions", comparison = False, return_fig = False, logits = None):

  a,b,c = calc_all_logit_diffs(cache)
  if logits is not None:
    ld = logits_to_ave_logit_diff(logits)
  else:
    ld = 0.00

  if not comparison:
    fig = imshow(
        torch.stack([a]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )
  else:

    ca, cb, cc = calc_all_logit_diffs(clean_cache)
    fig = imshow(
        torch.stack([a, a - ca]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"New Logit Diff - {ld:.2f}", "New Logit Diff - Clean Logit Diff",],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1000,
        margin={"r": 100, "l": 100}
    )


  if return_fig:
    return fig


In [77]:
heads =  [(9,9), (9,6), (10,0)]
model.reset_hooks() # callum library buggy
def return_item(item):
  return item

model.reset_hooks()
patched_logits = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = False
)

model.reset_hooks()
noise_patching_results = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = partial(display_corrupted_clean_logits, title = f"Logits When Sample Ablating in NMHs", comparison = True, logits = patched_logits),
    verbose = False,
    apply_metric_to_cache = True
)

Tried to stack head results when they weren't cached. Computing head results now


In [78]:
neg_m_heads = [(10,7), (11,10)]
name_mover_heads = [(9,9), (9,6), (10,0)]
backup_heads = [(9,0), (9,7), (10,1), (10,2), (10,6), (10,10), (11,2), (11,9)]
key_backup_heads = [(10,2), (10,10), (11,2)]
strong_neg_backup_heads = [(11,2), (10,2), (10,0), (11,6)]



head_names = ["Negative", "Name Mover", "Backup"]
head_list = [neg_m_heads, name_mover_heads, backup_heads]

In [79]:
def store_activation(
    activation,
    hook: HookPoint,
    where_to_store
):
    """
    takes a storage container where_to_store, and stores the activation in it at a hook
    """""
    where_to_store[:] = activation

In [80]:
def kq_rewrite_hook(
    internal_value: Float[Tensor, "batch seq head d_head"],
    hook: HookPoint,
    head,
    unnormalized_resid:  Float[Tensor, "batch seq d_model"],
    vector,
    act_name,
    scale = 1,
    position = -1,
    pre_ln = True
):
  """
  replaces keys or queries with a new result which we get from adding a vector to a position at the residual stream
  head: tuple for head to rewrite keys for
  unnormalized_resid: stored unnormalized residual stream needed to recalculated activations
  """

  ln1 = model.blocks[hook.layer()].ln1
  temp_resid = unnormalized_resid.clone()

  if pre_ln:
    temp_resid[:, position, :] = temp_resid[:, position, :] + scale * vector
    normalized_resid = ln1(temp_resid)
  else:
    temp_resid = ln1(temp_resid)
    temp_resid[:, position, :] = temp_resid[:, position, :] + scale * vector
    normalized_resid = temp_resid


  assert act_name == "q" or act_name == "k"
  if act_name == "q":
    W_Q, b_Q = model.W_Q[head[0], head[1]], model.b_Q[head[0], head[1]]
    internal_value[..., head[1], :] = einops.einsum(normalized_resid, W_Q, "batch seq d_model, d_model d_head -> batch seq d_head") + b_Q

  elif act_name == "k":
    W_K, b_K = model.W_K[head[0], head[1]], model.b_K[head[0], head[1]]
    internal_value[..., head[1], :] = einops.einsum(normalized_resid, W_K, "batch seq d_model, d_model d_head -> batch seq d_head") + b_K


In [81]:
def patch_head_vector(
    head_vector: Float[Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    head_indices: int,
    other_cache: ActivationCache
) -> Float[Tensor, "batch pos head_index d_head"]:
    '''
    Patches the output of a given head (before it's added to the residual stream) at
    every sequence position, using the value from the other cache.
    '''
    for head_index in head_indices:
      head_vector[:, :, head_index] = other_cache[hook.name][:, :, head_index]
    return head_vector

In [82]:
def patch_ln_scale(ln_scale, hook):
  #print(torch.equal(ln_scale, clean_cache["blocks." + str(hook.layer()) + ".ln1.hook_scale"]))
  ln_scale = clean_cache["blocks." + str(hook.layer()) + ".ln1.hook_scale"]
  return ln_scale

In [83]:
def causal_write_into_component(act_comp, head, direction, x, pre_ln = True, result_cache_function = None, result_cache_fun_has_head_input = False, freeze_layernorm = False, ablate_heads = []):
  '''
  writes a vector into the component at a given head
  returns new logit differences of run by default, or pass result_cache_funciton to run on cache

  head - tuple for head to intervene in act_comp for
  direction - vector to add to the act_comp in the head
  x - tensor of amount to scale
  '''
  y = torch.zeros(x.shape)
  for i in range(len(x)):
    scale = x[i]
    model.reset_hooks()
    temp = torch.zeros((batch_size, seq_len, model.cfg.d_model)).cuda()
    model.add_hook(utils.get_act_name("resid_pre", head[0]), partial(store_activation, where_to_store = temp))
    if freeze_layernorm:
      model.add_hook("blocks." + str(head[0]) + ".ln1.hook_scale", patch_ln_scale)
    model.add_hook(utils.get_act_name(act_comp, head[0]), partial(kq_rewrite_hook, head = head, unnormalized_resid = temp, vector = direction, act_name = act_comp, scale = scale, pre_ln = pre_ln))


    if len(ablate_heads) != 0:
      for j in ablate_heads:
        model.add_hook(utils.get_act_name("z", j[0]), partial(patch_head_vector, head_indices = [j[1]], other_cache = corrupted_cache))


    hooked_logits, hooked_cache = model.run_with_cache(clean_tokens)
    model.reset_hooks()


    if result_cache_function != None:
      if not result_cache_fun_has_head_input:
        y[i] = result_cache_function(hooked_cache)
      else:
        y[i] = result_cache_function(hooked_cache, head)
    else:
      # just calculate logit diff
      y[i] = logits_to_ave_logit_diff(hooked_logits)

  return y


In [84]:
def graph_lines(results, heads, x, title = "Effect of adding/subtracting direction", xtitle = "Scaling on direction", ytitle = "Logit Diff"):
  fig = px.line(title = title)
  for i in range(len(results)):
    fig.add_trace(go.Scatter(x = x, y = results[i], name = str(heads[i])))

  fig.update_xaxes(title = xtitle)
  fig.update_yaxes(title = ytitle)
  fig.show()

In [85]:
def get_head_IO_minus_S_attn(cache, head):

  layer, h_index = head

  attention_patten = cache["pattern", layer]
  S1 = attention_patten.mean(0)[h_index][-1][2].item()
  IO = attention_patten.mean(0)[h_index][-1][4].item()
  S2 = attention_patten.mean(0)[h_index][-1][10].item()

  return IO - S1 - S2

def get_head_attn(cache, head, token):

  layer, h_index = head

  attention_patten = cache["pattern", layer]

  if token == "S1":
     return attention_patten.mean(0)[h_index][-1][2].item()
  elif token == "IO":
     return attention_patten.mean(0)[h_index][-1][4].item()
  elif token == "S2":
     return attention_patten.mean(0)[h_index][-1][10].item()
  elif token == "BOS":
     return attention_patten.mean(0)[h_index][-1][0].item()
  else:
    print("RAHHHHH YOU MISSTYPED SOMETHING")

def patch_head_vector(
    head_vector: Float[Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    head_indices: int,
    other_cache: ActivationCache
) -> Float[Tensor, "batch pos head_index d_head"]:
    '''
    Patches the output of a given head (before it's added to the residual stream) at
    every sequence position, using the value from the other cache.
    '''
    for head_index in head_indices:
      head_vector[:, :, head_index] = other_cache[hook.name][:, :, head_index]
    return head_vector

def get_attn_results_into_head_dirs(heads, direction, scale_amounts, ablate_heads = []):
  io_attn_postln_nmh_results = []
  for i in range(len(heads)):
    io_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, result_cache_function = partial(get_head_attn, token = "IO"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))


  s1_attn_postln_nmh_results = []
  for i in range(len(heads)):
    s1_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, result_cache_function = partial(get_head_attn, token = "S1"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  s2_attn_postln_nmh_results = []
  for i in range(len(heads)):
    s2_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, result_cache_function = partial(get_head_attn, token = "S2"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  diff_results = []
  for i in range(len(heads)):
    diff_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, result_cache_function = get_head_IO_minus_S_attn, result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))


  bos_attn_postln_nmh_results = []
  for i in range(len(heads)):
    bos_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, result_cache_function = partial(get_head_attn, token = "BOS"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  return [io_attn_postln_nmh_results, s1_attn_postln_nmh_results, s2_attn_postln_nmh_results, diff_results, bos_attn_postln_nmh_results]

In [86]:
IO_unembed_direction = model.W_U.T[clean_tokens][:, 4, :]

In [87]:
def get_projection_onto(a, b):
  '''
  gets projection of a onto b
  '''
  dot_product = einops.einsum(a, b, "batch d_model, batch d_model -> batch")
  length_of_vector = einops.einsum(b, b, "batch d_model, batch d_model -> batch")
  projected_lengths = dot_product / length_of_vector
  return b * einops.repeat(projected_lengths, "batch -> batch d_model", d_model = model.cfg.d_model)


def get_io_and_non_io(direction):
 
  io_in_9_9_direction = get_projection_onto(direction, IO_unembed_direction)
  everything_but_9_9_direction = direction - io_in_9_9_direction
  return io_in_9_9_direction, everything_but_9_9_direction

# Looking into GPT-2 Backup

In [88]:
W_O_9_9 = model.W_O[9,9]

layer_nine_z = clean_cache[utils.get_act_name("z", 9)]
layer_nine_result = einops.einsum(W_O_9_9, layer_nine_z, "d_head d_model, batch seq h_idx d_head -> batch seq h_idx d_model")
output_9_9 = layer_nine_result[:, -1, 9, :]     #[:, 10, :]  # 10 = Subject,   4 = IO

_, non_9_9_io = get_io_and_non_io(output_9_9)

W_O_9_6 = model.W_O[9,6]
layer_nine_z = clean_cache[utils.get_act_name("z", 9)]
layer_nine_result = einops.einsum(W_O_9_6, layer_nine_z, "d_head d_model, batch seq h_idx d_head -> batch seq h_idx d_model")
output_9_6 = layer_nine_result[:, -1, 6, :]     #[:, 10, :]  # 10 = Subject,   4 = IO

_, non_9_6_io = get_io_and_non_io(output_9_6)

In [89]:
overlap_dir = get_projection_onto(non_9_9_io, non_9_6_io)

In [98]:
scaling_amounts = torch.linspace(-20, 20, 101)
overlap_into_backup = get_attn_results_into_head_dirs(top_neg_heads, overlap_dir, scaling_amounts)

In [100]:
fig = make_subplots(rows = 1, cols = 5, subplot_titles = ["IO", "S1", "S2","IO - S1 - S2", "BOS"], shared_yaxes=True)
fig.update_layout(height = 400)

fig.update_layout(title = "GPT-2 Small Effect of adding/subtracting components pre_ln to Backup Heads while NMHs are Ablated")
colors = [
    "pink", "darkviolet", "blue", "purple", "turquoise", "red", "green", "yellow", "orange", "cyan", "magenta",
    "lime", "maroon", "navy", "olive", "teal", "aqua", "silver", "gray", "black", "white", "indigo", "gold", "brown",
    "coral", "crimson", "darkblue", "darkcyan", "darkgoldenrod", "darkgray", "darkgreen", "darkkhaki", "darkmagenta",
    "darkolivegreen", "darkorange", "darkorchid", "darkred", "darksalmon", "darkslateblue", "darkslategray",
    "darkturquoise", "darkviolet", "deeppink", "deepskyblue", "dimgray", "dodgerblue", "firebrick", "forestgreen",
    "fuchsia", "gainsboro", "gold", "goldenrod", "gray", "greenyellow", "honeydew", "hotpink", "indianred", "indigo",
    "ivory", "khaki", "lavender", "lavenderblush", "lawngreen", "lemonchiffon", "lightblue", "lightcoral",
    "lightcyan", "lightgoldenrodyellow", "lightgray", "lightgreen", "lightpink", "lightsalmon", "lightseagreen",
    "lightskyblue", "lightslategray", "lightsteelblue", "lightyellow", "lime", "limegreen", "linen", "magenta",
    "maroon", "mediumaquamarine", "mediumblue", "mediumorchid", "mediumpurple", "mediumseagreen", "mediumslateblue",
    "mediumspringgreen", "mediumturquoise", "mediumvioletred", "midnightblue", "mintcream", "mistyrose", "moccasin",
    "navajowhite", "navy", "oldlace", "olive", "olivedrab", "orangered", "orchid", "palegoldenrod", "palegreen",
    "paleturquoise", "palevioletred", "papayawhip", "peachpuff", "peru", "plum", "powderblue", "rebeccapurple",
    "rosybrown", "royalblue", "saddlebrown", "salmon", "sandybrown", "seagreen", "sienna", "skyblue", "slateblue",
    "slategray", "snow", "springgreen", "steelblue", "tan", "thistle", "tomato", "turquoise", "violet", "wheat",
    "whitesmoke", "yellowgreen"
]


for col, data in enumerate(overlap_into_backup):
  for i in range(len(data)):
    fig.add_trace(go.Scatter(x = scaling_amounts, y = data[i], name = str(top_neg_heads[i]), line_color = colors[i], legendgroup= str(i)), row = 1, col = col + 1)


fig.update_xaxes(title = "Scaling on (9.9 ∥ 9.6) ⊥", row = 1)
fig.update_yaxes(title = "Attn to token")
fig.show()